In [1]:
# =========================
# 0 - Imports
# =========================

# Packages
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
import cartopy.io.img_tiles as cimgt
import cmocean
import cool_maps.plot as cplt
import dask
from datetime import datetime
import PIL as pil
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import os
import pandas as pd
import xarray as xr

# Local Functions
from sup_functions import check_abort, check_float, check_coordinate
from sup_functions import calculate_distance, calculate_heading, DD_to_DDM
from sup_plots import calculate_ticks, DD_to_DM
from sup_slocum import goto_l10

In [2]:
config = {
    "glider_name": "Yucatan", #
    "max_depth": 1000.0, #
    "avg_velocity": 0.5, #
    "battery_capacity": 3000, # 3x G3 for Sentinel
    "battery_drain": 5, #
    "satisfying_radius": 1000, #
    "waypoints": [(15.365189027409214, -90.57125927041142),
                    (23.25501019140833, -80.12008055595216)
                ] # Focused Yucatan
}

waypoints = config["waypoints"]

In [3]:
class RTOFS():

    ### FUNCTION:
    def __init__(self) -> None:
        
        """
        Initialize the RTOFS instance and fetch initial RTOFS data.

        Args:
        - None

        Returns:
        - None
        """
        
        self._data_orig = self.rtofs_load()
        self._data_orig = self._data_orig.set_coords(['lat', 'lon'])
        self.data = self._data_orig.copy()
        self.x = self.data.x.values
        self.y = self.data.y.values
        self.grid_lons = self.data.lon.values[0,:]
        self.grid_lats = self.data.lat.values[:,0]
    
    ### FUNCTION:
    def rtofs_load(self):
        
        """
        Fetch the RTOFS data from the given URL and set its coordinates.
        
        Args:
        - None

        Returns:
        - rtofs_dataset (xarray.Dataset): RTOFS data
        """
        
        rtofs_access = "https://tds.marine.rutgers.edu/thredds/dodsC/cool/rtofs/rtofs_us_east_scraped"
        # rtofs_access = "rtofs_us_east_scraped.nc"
        
        try:
            rtofs_dataset = xr.open_dataset(rtofs_access).set_coords(['lon', 'lat'])
            rtofs_dataset.attrs['model'] = 'RTOFS'
            rtofs_dataset= rtofs_dataset.isel(time=-1)
            return rtofs_dataset
        except Exception as e:
            print(f"Error fetching RTOFS data: {e}")
            return None
    
    ### FUNCTION:
    def rtofs_subset(self, config, waypoints, buffer=0.5, subset=True):
        
        """
        Subset the RTOFS data based on the bounding box created by the given points.

        Args:
        - config (dict): Glider Guidance System configuration
        - waypoints (list): list of waypoints
        - buffer (float): buffer to add to the bounding box
        - subset (bool): whether or not to subset the data

        Returns:
        - None
        """

        if subset:
            lats, lons = zip(*waypoints)

            min_lon, max_lon = min(lons) - buffer, max(lons) + buffer
            min_lat, max_lat = min(lats) - buffer, max(lats) + buffer

            lons_ind = np.interp([min_lon, max_lon], self.grid_lons, self.x)
            lats_ind = np.interp([min_lat, max_lat], self.grid_lats, self.y)

            extent = [
                np.floor(lons_ind[0]).astype(int),
                np.ceil(lons_ind[1]).astype(int),
                np.floor(lats_ind[0]).astype(int),
                np.ceil(lats_ind[1]).astype(int)
            ]

            self.data = self._data_orig.isel(
                x=slice(extent[0], extent[1]),
                y=slice(extent[2], extent[3])
            )
        
            self.data_lons = self.data.lon.values[0,:]
            self.data_lats = self.data.lat.values[:,0]

            self.data = self.data.where(self.data['depth'] <= config["max_depth"], drop=True)

        else:
            self.data = self._data_orig

            self.data_lons = self.data.lon.values[0, :]
            self.data_lats = self.data.lat.values[:, 0]

            self.data = self.data.where(self.data['depth'] <= config["max_depth"], drop=True)

RTOFS_class = RTOFS()

In [7]:
print(RTOFS_class.data)

<xarray.Dataset>
Dimensions:      (depth: 40, y: 1710, x: 742)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 1 2 3 4 5 6 7 ... 1704 1705 1706 1707 1708 1709 1710
  * x            (x) int32 1 2 3 4 5 6 7 8 9 ... 735 736 737 738 739 740 741 742
    time         datetime64[ns] 2023-11-10
    lat          (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 79.65 79.68 79.72 79.75
    lon          (y, x) float32 -100.0 -99.92 -99.84 ... -91.87 -91.82 -91.77
Data variables:
    date         float64 ...
    u            (depth, y, x) float32 ...
    v            (depth, y, x) float32 ...
    temperature  (depth, y, x) float32 ...
    salinity     (depth, y, x) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    re